In [58]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os


El objetivo de este Notebook es preparar los datos para modelar con redes neuronales

In [59]:
# Reducir memoria automáticamente
def optimizar_memoria(df):
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [60]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [61]:
df_full.shape

(17021654, 76)

In [62]:
# Buscar en df_full los product_id, customer_id que solo tienen ceros en TN
def buscar_productos_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    grouped = df.groupby(['PRODUCT_ID', 'CUSTOMER_ID'])['TN'].sum().reset_index()
    productos_solo_ceros = grouped[grouped['TN'] == 0]
    return productos_solo_ceros

productos_solo_ceros = buscar_productos_solo_ceros(df_full)
print(f"🔍 Combinaciones PRODUCT_ID + CUSTOMER_ID con TN = 0 en todos sus registros: {len(productos_solo_ceros)}")

# Eliminar del df_full los product_id, customer_id que solo tienen ceros en TN
def eliminar_productos_solo_ceros(df: pd.DataFrame, productos_solo_ceros: pd.DataFrame) -> pd.DataFrame:
    productos_set = set(zip(productos_solo_ceros['PRODUCT_ID'], productos_solo_ceros['CUSTOMER_ID']))
    mask = df.set_index(['PRODUCT_ID', 'CUSTOMER_ID']).index.isin(productos_set)
    
    cantidad_eliminada = mask.sum()
    print(f"🗑️ Filas eliminadas de df_full: {cantidad_eliminada:,}")
    
    df_filtrado = df[~mask]
    return df_filtrado

df_full = eliminar_productos_solo_ceros(df_full, productos_solo_ceros)


🔍 Combinaciones PRODUCT_ID + CUSTOMER_ID con TN = 0 en todos sus registros: 327068
🗑️ Filas eliminadas de df_full: 6,594,430


In [63]:
# Eliminar de df_full las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [64]:
df_full.shape

(7781619, 75)

In [65]:
# Por cada PERIODO mostrar la cantidad de filas
periodo_counts = df_full['PERIODO'].value_counts().sort_index()
print("📅 Cantidad de filas por PERIODO:")
print(periodo_counts)


📅 Cantidad de filas por PERIODO:
PERIODO
201701    153506
201702    169292
201703    173972
201704    175629
201705    177198
201706    179909
201707    184384
201708    187774
201709    190994
201710    195034
201711    201377
201712    201427
201801    201965
201802    202278
201803    203591
201804    207881
201805    212721
201806    213622
201807    215649
201808    216883
201809    222730
201810    226473
201811    229728
201812    230059
201901    230063
201902    230976
201903    235674
201904    244108
201905    247448
201906    251593
201907    256940
201908    260099
201909    262500
201910    262616
201911    262721
201912    262805
Name: count, dtype: int64


In [66]:
# Agregar una columna que indique la diferencia en ORDINAL entre el ORDINAL actual y el ORDINAL anterior donde TN sea mayor a 0
# para ese CUSTOMER_ID y PRODUCT_ID

from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_grupo(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_parallel(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = 36  # valor inicial
    df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'].astype('int16')

    # Agrupar por cliente y producto
    grupos = list(df.groupby(['CUSTOMER_ID', 'PRODUCT_ID']))

    # Procesar en paralelo
    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_grupo)(grupo) for _, grupo in grupos
    )

    # Concatenar todos los resultados
    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado



df_full = agregar_diferencia_ordinal_parallel(df_full, n_jobs=28)

In [67]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_producto(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_PRODUCT_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_por_producto(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = 36
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = df['MESES_SIN_COMPRAR_PRODUCT_ID'].astype('int16')

    # Agrupar solo por PRODUCT_ID
    grupos = list(df.groupby('PRODUCT_ID'))

    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_producto)(grupo) for _, grupo in grupos
    )

    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado

df_full = agregar_diferencia_ordinal_por_producto(df_full, n_jobs=28)


In [68]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_cliente(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_CUSTOMER_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_por_cliente(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_CUSTOMER_ID'] = 36
    df['MESES_SIN_COMPRAR_CUSTOMER_ID'] = df['MESES_SIN_COMPRAR_CUSTOMER_ID'].astype('int16')

    grupos = list(df.groupby('CUSTOMER_ID'))

    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_cliente)(grupo) for _, grupo in grupos
    )

    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado

df_full = agregar_diferencia_ordinal_por_cliente(df_full, n_jobs=28)

In [69]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
# Calcular los días del mes usando las columnas ANIO y MES

# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si ANIO==2019 y MES en [6, 8, 10], y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = np.where(
       (df_full['ANIO'] == 2019) & (df_full['MES'].isin([6, 8, 10])),
       1., 0.0
)
df_full['MES_PROBLEMATICO'] = df_full['MES_PROBLEMATICO'].astype(np.float32)

In [70]:
# Guardar el DataFrame en parquet
df_full.to_parquet('./data/interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [71]:

# Conservar las siguientes columnas
columns_to_keep = ['MES_SIN', 'MES_COS', 'ID_CAT1',
       'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID',
       'PRODUCT_ID', 'CUST_REQUEST_QTY',
       'CUST_REQUEST_TN', 'TN', 'CLASE_DELTA',
       'ANTIG_CLIENTE','ANTIG_PRODUCTO', 'CANT_PROD_CLI_PER',
       'MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID','MESES_SIN_COMPRAR_PRODUCT_ID','MESES_SIN_COMPRAR_CUSTOMER_ID',
       'MES_PROBLEMATICO','PERIODO','ORDINAL']
# Filtrar el DataFrame para conservar solo las columnas deseadas 
df_full = df_full[columns_to_keep]

In [72]:
def log_transform_signed(x):
    return np.sign(x) * np.log1p(np.abs(x))

df_full['CLASE_DELTA_LOG1P'] = log_transform_signed(df_full['CLASE_DELTA'])


In [73]:
from sklearn.preprocessing import StandardScaler

scaler_y = StandardScaler()
df_full['CLASE_DELTA_LOG1P_Z'] = scaler_y.fit_transform(df_full[['CLASE_DELTA_LOG1P']])

media_y = scaler_y.mean_[0]
std_y = scaler_y.scale_[0]

print(f"Media de CLASE_DELTA_LOG1P_Z: {media_y}")
print(f"Desviación estándar de CLASE_DELTA_LOG1P_Z: {std_y}")



Media de CLASE_DELTA_LOG1P_Z: -0.0002512964473318228
Desviación estándar de CLASE_DELTA_LOG1P_Z: 0.2535427832171512


In [74]:
def revert_clase_delta_log1p_z(pred_z, media = media_y, std = std_y):
    pred_log1p = pred_z * std + media
    return np.sign(pred_log1p) * (np.expm1(np.abs(pred_log1p)))


In [75]:
# Eliminar las columnas CLASE_DELTA y CLASE_DELTA_LOG1P para evitar confusiones
df_full.drop(columns=['CLASE_DELTA', 'CLASE_DELTA_LOG1P'], inplace=True, errors='ignore')

In [76]:


cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE']

for col in cat_cols:
    if df_full[col].isnull().any():
        df_full[col] = df_full[col].fillna("missing")


encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_full.loc[:, col] = le.fit_transform(df_full[col]).astype(int)
    encoders[col] = le  # para guardar los mapeos por si necesitás revertirlos


In [77]:
# Guardo los encoders en archivos .pkl
import joblib
import os

os.makedirs('encoders', exist_ok=True)

for col, le in encoders.items():
    joblib.dump(le, f'encoders/{col}_encoder.pkl')

In [78]:
import numpy as np
import pandas as pd

def generar_lags_deltas_pct(
    df: pd.DataFrame,
    columnas: list,
    lags: list = list(range(1, 4)),  # por default lag 1, 2, 3
    group_cols: list = ['PRODUCT_ID', 'CUSTOMER_ID'],
    time_col: str = 'ORDINAL'
):
    df = df.sort_values(by=group_cols + [time_col])
    nuevas_columnas = []

    for col in columnas:
        # Para evitar división por 0: obtener mínimo positivo por grupo
        minimos = df.groupby(group_cols)[col].transform('min')

        for lag in lags:
            lag_col = f'{col}_LAG_{lag:02d}'
            delta_col = f'{col}_DELTA_{lag:02d}'
            delta_pct_col = f'{col}_DELTA_PCT_{lag:02d}'

            # LAG
            df[lag_col] = df.groupby(group_cols)[col].shift(lag).fillna(0)
            nuevas_columnas.append(lag_col)

            # DELTA
            df[delta_col] = df[col] - df[lag_col]
            nuevas_columnas.append(delta_col)

            # DELTA PCT
            reemplazo = np.where(df[lag_col] == 0, minimos, df[lag_col])
            df[delta_pct_col] = df[col] / reemplazo
            df[delta_pct_col] = df[delta_pct_col].replace([np.inf, -np.inf], 0).fillna(0)
            nuevas_columnas.append(delta_pct_col)

    return df, nuevas_columnas


In [79]:
columnas_lags_deltas = [
    'TN',
    'CUST_REQUEST_QTY',
    'CUST_REQUEST_TN',
    'CANT_PROD_CLI_PER'
]

df_full, nuevas_col_lags_deltas = generar_lags_deltas_pct(
    df=df_full,
    columnas=columnas_lags_deltas,
    lags=list(range(1, 19))  # como hacías con TN
)

/tmp/ipykernel_377801/3165156112.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col] = df.groupby(group_cols)[col].shift(lag).fillna(0)
/tmp/ipykernel_377801/3165156112.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[delta_col] = df[col] - df[lag_col]
/tmp/ipykernel_377801/3165156112.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To g

In [80]:
# Excluir la variable de clase
cols_to_check = df_full.columns.difference(['CLASE_DELTA_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_full[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [81]:
print(f"📊 DataFrame final con {df_full.shape[0]:,} filas y {df_full.shape[1]} columnas:")

📊 DataFrame final con 7,781,619 filas y 238 columnas:


In [82]:
# Copia de seguridad
df = df_full.copy()

# === Normalización ===
scaler_customer = StandardScaler()
df['CUSTOMER_RANK_NORM'] = scaler_customer.fit_transform(df[['CUSTOMER_ID']])
joblib.dump(scaler_customer, './encoders/scaler_customer_id.pkl')

# === Binning (en deciles) ===
df['CUSTOMER_RANK_BIN'] = pd.qcut(df['CUSTOMER_ID'], q=10, labels=False)
df['CUSTOMER_RANK_BIN'] = df['CUSTOMER_RANK_BIN'].astype('category')

# Validación opcional
assert df['CUSTOMER_RANK_BIN'].isna().sum() == 0, "NaNs en qcut"

# Reemplazar en df_full
df_full = df



In [83]:
# Copia de seguridad del DataFrame
df = df_full.copy()

# === Normalización ===
scaler_product = StandardScaler()
df['PRODUCT_RANK_NORM'] = scaler_product.fit_transform(df[['PRODUCT_ID']])
joblib.dump(scaler_product, './encoders/scaler_product_id.pkl')

# === Binning (en deciles) ===
df['PRODUCT_RANK_BIN'] = pd.qcut(df['PRODUCT_ID'], q=10, labels=False)
df['PRODUCT_RANK_BIN'] = df['PRODUCT_RANK_BIN'].astype('category')

# Validación opcional
assert df['PRODUCT_RANK_BIN'].isna().sum() == 0, "NaNs en qcut de PRODUCT_ID"

# Reemplazar en el DataFrame principal
df_full = df


In [84]:
print(df_full.columns.tolist())

['MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'TN', 'ANTIG_CLIENTE', 'ANTIG_PRODUCTO', 'CANT_PROD_CLI_PER', 'MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID', 'MESES_SIN_COMPRAR_PRODUCT_ID', 'MESES_SIN_COMPRAR_CUSTOMER_ID', 'MES_PROBLEMATICO', 'PERIODO', 'ORDINAL', 'CLASE_DELTA_LOG1P_Z', 'TN_LAG_01', 'TN_DELTA_01', 'TN_DELTA_PCT_01', 'TN_LAG_02', 'TN_DELTA_02', 'TN_DELTA_PCT_02', 'TN_LAG_03', 'TN_DELTA_03', 'TN_DELTA_PCT_03', 'TN_LAG_04', 'TN_DELTA_04', 'TN_DELTA_PCT_04', 'TN_LAG_05', 'TN_DELTA_05', 'TN_DELTA_PCT_05', 'TN_LAG_06', 'TN_DELTA_06', 'TN_DELTA_PCT_06', 'TN_LAG_07', 'TN_DELTA_07', 'TN_DELTA_PCT_07', 'TN_LAG_08', 'TN_DELTA_08', 'TN_DELTA_PCT_08', 'TN_LAG_09', 'TN_DELTA_09', 'TN_DELTA_PCT_09', 'TN_LAG_10', 'TN_DELTA_10', 'TN_DELTA_PCT_10', 'TN_LAG_11', 'TN_DELTA_11', 'TN_DELTA_PCT_11', 'TN_LAG_12', 'TN_DELTA_12', 'TN_DELTA_PCT_12', 'TN_LAG_13', 'TN_DELTA_13', 'TN_DELTA_PCT_13', 'TN_LAG_

In [85]:
# Copia del DataFrame original
df = df_full.copy()

# Columnas categóricas
cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE','CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN']

# Columnas a excluir
excluir = ['PERIODO','CLASE_DELTA_LOG1P_Z', 'CUSTOMER_RANK_NORM', 'PRODUCT_RANK_NORM', 
           'MES_SIN', 'MES_COS', 'MES_PROBLEMATICO', 'CUSTOMER_ID','PRODUCT_ID'] + cat_cols
excluir += [col for col in df.columns if col.endswith('_BIN')]

# Columnas numéricas a escalar
cols_a_escalar = [col for col in df.columns if col not in excluir and pd.api.types.is_numeric_dtype(df[col])]

# Entrenamiento del scaler SOLO con datos de entrenamiento
df_entrenamiento = df[df['PERIODO'] <= 201910].copy()
scaler = StandardScaler()
scaler.fit(df_entrenamiento[cols_a_escalar])

# Aplicación del scaler a TODO el dataset
valores_escalados = scaler.transform(df[cols_a_escalar])
df_scaled = pd.DataFrame(valores_escalados, columns=[col + '_Z' for col in cols_a_escalar], index=df.index)

# Reemplazo de columnas originales
df.drop(columns=cols_a_escalar, inplace=True)
df = pd.concat([df, df_scaled], axis=1)

# Guardar scaler
joblib.dump(scaler, './encoders/scaler_features_numericas.pkl')

# Actualizar df_full
df_full = df


In [86]:
print(df_full.columns.tolist())

['MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'MES_PROBLEMATICO', 'PERIODO', 'CLASE_DELTA_LOG1P_Z', 'CUSTOMER_RANK_NORM', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_NORM', 'PRODUCT_RANK_BIN', 'CUST_REQUEST_QTY_Z', 'CUST_REQUEST_TN_Z', 'TN_Z', 'ANTIG_CLIENTE_Z', 'ANTIG_PRODUCTO_Z', 'CANT_PROD_CLI_PER_Z', 'MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID_Z', 'MESES_SIN_COMPRAR_PRODUCT_ID_Z', 'MESES_SIN_COMPRAR_CUSTOMER_ID_Z', 'ORDINAL_Z', 'TN_LAG_01_Z', 'TN_DELTA_01_Z', 'TN_DELTA_PCT_01_Z', 'TN_LAG_02_Z', 'TN_DELTA_02_Z', 'TN_DELTA_PCT_02_Z', 'TN_LAG_03_Z', 'TN_DELTA_03_Z', 'TN_DELTA_PCT_03_Z', 'TN_LAG_04_Z', 'TN_DELTA_04_Z', 'TN_DELTA_PCT_04_Z', 'TN_LAG_05_Z', 'TN_DELTA_05_Z', 'TN_DELTA_PCT_05_Z', 'TN_LAG_06_Z', 'TN_DELTA_06_Z', 'TN_DELTA_PCT_06_Z', 'TN_LAG_07_Z', 'TN_DELTA_07_Z', 'TN_DELTA_PCT_07_Z', 'TN_LAG_08_Z', 'TN_DELTA_08_Z', 'TN_DELTA_PCT_08_Z', 'TN_LAG_09_Z', 'TN_DELTA_09_Z', 'TN_DELTA_PCT_09_Z', 'TN_LAG_10_Z', 'TN_DELTA_10_Z', 'TN_DE

In [87]:
# Excluir la variable de clase
cols_to_check = df_full.columns.difference(['CLASE_DELTA_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_full[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [88]:
print(df_full.shape)
# Guardar el DataFrame resultante en un archivo parquet
df_full.to_parquet('./data/train_val_NN_TORCH.parquet', engine='fastparquet', index=False)

(7781619, 242)
